In [33]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob


PATH_TO_DATA = '/home/lgierz/data/'

model_names = ["Linear Classifier", "KNN"]
fm_names = ['resnet', 'dino']

dataset_configs = {
    'top277': (277, [3000, 2000, 1000, 500]),
    'top387': (387, [2000, 1000, 500]),
    'top589': (589, [1000, 500])
}
PATH_TO_CA = PATH_TO_DATA + 'run_results/confidence_tests/'

PATH_TO_IMAGES = '/mnt/data/lgierz/moth_dataset_top589_max3000/'
PATH_TO_LABELS = '/home/lgierz/BA_MothClassification/data_big_files/dataset_full_top589_max3000.csv'
PATH_TO_CORRECTED_CONFS = PATH_TO_CA + 'all_confidences_results.csv'

labels_csv = pd.read_csv(PATH_TO_LABELS)
confidences_csv = pd.read_csv(PATH_TO_CORRECTED_CONFS)

# create real mapping for labels_csv
species = labels_csv['scientificName']
unique_species = sorted(species.unique())
species_mapping = {species: idx for idx, species in enumerate(unique_species)}

print(unique_species)
print(species_mapping)
print(len(unique_species))

/tmp/ipykernel_145335/2977717358.py:24: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  labels_csv = pd.read_csv(PATH_TO_LABELS)


['Abraxas grossulariata', 'Abraxas sylvata', 'Abrostola triplasia', 'Acasis viretata', 'Acentria ephemerella', 'Acleris forsskaleana', 'Acleris notana/ferrugana', 'Acleris variegana', 'Acontia lucida', 'Acontia trabealis', 'Acrobasis advenella', 'Acrobasis repandana', 'Acrobasis tumidana', 'Acronicta aceris', 'Acronicta leporina', 'Acronicta rumicis', 'Acronicta tridens / psi', 'Adela reaumurella', 'Adscita statices', 'Aethalura punctulata', 'Agapeta hamana', 'Agapeta zoegana', 'Aglais io', 'Aglais urticae', 'Aglossa pinguinalis', 'Agonopterix arenella', 'Agonopterix heracliana / ciliella', 'Agriopis aurantiaria', 'Agriopis leucophaearia', 'Agriopis marginaria', 'Agriphila geniculea', 'Agriphila inquinatella', 'Agriphila selasella', 'Agriphila straminella', 'Agriphila tristella', 'Agrius convolvuli', 'Agrochola circellaris', 'Agrochola lota', 'Agrochola lunosa', 'Agrochola lychnidis', 'Agrochola macilenta', 'Agrotis clavis', 'Agrotis exclamationis', 'Agrotis ipsilon', 'Agrotis puta', '

In [34]:
# Create mappings from species name to family and from species name to genus
species2family = labels_csv[['scientificName','family']].drop_duplicates().set_index('scientificName')['family'].to_dict()

label_family_mapping = {}
label_genus_mapping = {}
for sp, lbl in species_mapping.items():
    label_family_mapping[lbl] = species2family.get(sp, 'Unknown')
    label_genus_mapping[lbl] = sp.split(' ')[0]

print(len(set(label_family_mapping.values())), ' ', end='')
print(label_family_mapping)

print(len(set(label_genus_mapping.values())), ' ', end='')
print(label_genus_mapping)

40  {0: 'Geometridae', 1: 'Geometridae', 2: 'Noctuidae', 3: 'Geometridae', 4: 'Crambidae', 5: 'Tortricidae', 6: 'Tortricidae', 7: 'Tortricidae', 8: 'Noctuidae', 9: 'Noctuidae', 10: 'Pyralidae', 11: 'Pyralidae', 12: 'Pyralidae', 13: 'Noctuidae', 14: 'Noctuidae', 15: 'Noctuidae', 16: 'Noctuidae', 17: 'Adelidae', 18: 'Zygaenidae', 19: 'Geometridae', 20: 'Tortricidae', 21: 'Tortricidae', 22: 'Nymphalidae', 23: 'Nymphalidae', 24: 'Pyralidae', 25: 'Depressariidae', 26: 'Depressariidae', 27: 'Geometridae', 28: 'Geometridae', 29: 'Geometridae', 30: 'Crambidae', 31: 'Crambidae', 32: 'Crambidae', 33: 'Crambidae', 34: 'Crambidae', 35: 'Sphingidae', 36: 'Noctuidae', 37: 'Noctuidae', 38: 'Noctuidae', 39: 'Noctuidae', 40: 'Noctuidae', 41: 'Noctuidae', 42: 'Noctuidae', 43: 'Noctuidae', 44: 'Noctuidae', 45: 'Noctuidae', 46: 'Noctuidae', 47: 'Geometridae', 48: 'Tortricidae', 49: 'Noctuidae', 50: 'Geometridae', 51: 'Alucitidae', 52: 'Erebidae', 53: 'Pterophoridae', 54: 'Noctuidae', 55: 'Noctuidae', 56: 

In [35]:
confidences_csv['FamilyCorrect'] = confidences_csv.apply(
    lambda row: label_family_mapping[row['Label']] == label_family_mapping[row['PredictedLabel']],
    axis=1
)

confidences_csv['GenusCorrect'] = confidences_csv.apply(
    lambda row: label_genus_mapping[row['Label']] == label_genus_mapping[row['PredictedLabel']],
    axis=1
)


In [36]:

confidences_csv['Top5Correct'] = confidences_csv.apply(
    lambda row: row['Label'] in [lab for lab, _ in eval(row['Confidence'])[:5]],
    axis=1
)

### Calculate Top1 Accuracy

In [41]:

acc = confidences_csv.groupby(['FoundationalModel', 'Model', 'Species', 'Samples'])['Correct'].mean().apply(lambda x: round(x*100, 2)).reset_index()
acc[((acc['Species'] == 277) & ((acc['Samples'] == 1000) | (acc['Samples'] == 3000)))  | ((acc['Species'] == 589)& (acc['Samples'] == 1000))].reset_index(drop=True)


,FoundationalModel,Model,Species,Samples,Correct
0,dino,KNN,277,1000,70.49
1,dino,KNN,277,3000,75.02
2,dino,KNN,589,1000,65.39
3,dino,Linear Classifier,277,1000,91.12
4,dino,Linear Classifier,277,3000,92.80
5,dino,Linear Classifier,589,1000,88.70
6,resnet,KNN,277,1000,40.69
7,resnet,KNN,277,3000,45.78
8,resnet,KNN,589,1000,33.90
9,resnet,Linear Classifier,277,1000,67.74


### Calculate Top5 Accuracy

In [42]:

top5acc = confidences_csv.groupby(['FoundationalModel', 'Model', 'Species', 'Samples'])['Top5Correct'].mean().apply(lambda x: round(x*100, 2)).reset_index()
top5acc[((acc['Species'] == 277) & ((acc['Samples'] == 1000) | (acc['Samples'] == 3000)))  | ((acc['Species'] == 589)& (acc['Samples'] == 1000))].reset_index(drop=True)

,FoundationalModel,Model,Species,Samples,Top5Correct
0,dino,KNN,277,1000,88.49
1,dino,KNN,277,3000,91.01
2,dino,KNN,589,1000,84.52
3,dino,Linear Classifier,277,1000,97.76
4,dino,Linear Classifier,277,3000,98.31
5,dino,Linear Classifier,589,1000,96.89
6,resnet,KNN,277,1000,67.31
7,resnet,KNN,277,3000,71.88
8,resnet,KNN,589,1000,58.29
9,resnet,Linear Classifier,277,1000,87.90


### Genus Accuracy

In [ ]:

genusacc = confidences_csv.groupby(['FoundationalModel', 'Model', 'Species', 'Samples'])['GenusCorrect'].mean().apply(lambda x: round(x*100, 2)).reset_index()
genusacc[((acc['Species'] == 277) & ((acc['Samples'] == 1000) | (acc['Samples'] == 3000)))  | ((acc['Species'] == 589)& (acc['Samples'] == 1000))].reset_index(drop=True)

,FoundationalModel,Model,Species,Samples,GenusCorrect
0,dino,KNN,277,1000,73.88
1,dino,KNN,277,3000,78.03
2,dino,KNN,589,1000,70.70
3,dino,Linear Classifier,277,1000,92.64
4,dino,Linear Classifier,277,3000,94.16
5,dino,Linear Classifier,589,1000,91.39
6,resnet,KNN,277,1000,44.25
7,resnet,KNN,277,3000,49.27
8,resnet,KNN,589,1000,39.41
9,resnet,Linear Classifier,277,1000,70.43


### Family Accuracies

In [ ]:

familyacc = confidences_csv.groupby(['FoundationalModel', 'Model', 'Species', 'Samples'])['FamilyCorrect'].mean().apply(lambda x: round(x*100, 2)).reset_index()
familyacc[((acc['Species'] == 277) & ((acc['Samples'] == 1000) | (acc['Samples'] == 3000)))  | ((acc['Species'] == 589)& (acc['Samples'] == 1000))].reset_index(drop=True)

,FoundationalModel,Model,Species,Samples,FamilyCorrect
0,dino,KNN,277,1000,89.14
1,dino,KNN,277,3000,91.15
2,dino,KNN,589,1000,88.46
3,dino,Linear Classifier,277,1000,96.99
4,dino,Linear Classifier,277,3000,97.67
5,dino,Linear Classifier,589,1000,96.69
6,resnet,KNN,277,1000,66.49
7,resnet,KNN,277,3000,70.24
8,resnet,KNN,589,1000,64.04
9,resnet,Linear Classifier,277,1000,84.38
